<a href="https://colab.research.google.com/github/sizhky/AutoTrain/blob/segmentation/nbs/segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup Data and Libraries

In [1]:
%%capture
try:
    from icevision.all import *
except:
    !pip install -U torch-snippets typer imgaug
    !wget https://raw.githubusercontent.com/airctic/icevision/master/install_colab.sh
    !bash install_colab.sh

In [ ]:
# Setup Data
# %%capture
%load_ext autoreload
%autoreload 2

from torch_snippets import *

# If running locally from AutoTrain/nbs
if not os.path.exists('/content') and P().resolve().stem != 'AutoTrain':
    %cd ..

# If running from colab
if os.path.exists('/content') and not os.path.exists('/content/AutoTrain'):
    !git clone https://github.com/sizhky/AutoTrain

# change present working directory
if not P().resolve().stem == 'AutoTrain':
    %cd AutoTrain

!git pull origin segmentation
!git checkout segmentation

if not os.path.exists('balloon.zip'):
    !wget --quiet https://www.dropbox.com/s/tur7gzmuuzb2vzf/balloon.zip -O balloon.zip
if not os.path.exists('balloon/data/'):
    makedir('balloon')
    unzip_file('balloon.zip', 'balloon/data/')

## Verify Config

In [4]:
!cat config_segmentation.ini

[project]
owner: sizhky
name: baloons
location: ${project.name}
model_directory: ${project.location}/models
num_classes: 1
classes: ["Balloon"]
type: segmentation

[architecture]
model_type: torchvision.mask_rcnn
backbone: resnet34_fpn
pretrained: True
size: 384
[architecture.extra_args]

[training]
; absolute path, or path relative to AutoTrain folder
dir: balloon/data/balloon
; folder should be exported from label studio, 
; in COCO format for segmentation, i.e., 
; it should have one images dir and one results.json file
images_dir: ${training.dir}/images
annotations_file: ${training.dir}/result.json
train_ratio: 0.8

[training.preprocess]
@augmentations: get_train_transforms
size: ${architecture.size}
presize: 512

[training.scheme]
epochs: 20
lr: 1e-4
freeze_epochs: 1
resume_training_from: ${project.model_directory}/model.pt
output_path: ${project.model_directory}/model.pt

[testing]
[testing.preprocess]
@augmentations: get_val_transforms
size: ${architecture.size}
presize: 512


In [ ]:
os.environ['CONFIG'] = 'config_segmentation.ini'
!python auto_train_segmentation/train.py find-best-learning-rate

INFO     - 
CLASSES INFERRED FROM balloon/data/balloon/result.json: <ClassMap: {'background': 0, 'balloon': 1}> | __main__:<module>:34
100%|████████████████████████████████████████| 255/255 [00:00<00:00, 518.04it/s]
INFO     - Autofixing records | icevision.parsers.parser:parse:136
100%|█████████████████████████████████████████| 13/13 [00:00<00:00, 5991.20it/s]
INFO     - LR Plot is saved at baloons/find_lr_plot.png | __main__:find_best_learning_rate:80
INFO     - Suggested LRs: 0.0006309573538601399 and 1.4454397387453355e-05 | __main__:find_best_learning_rate:81


In [33]:
os.environ['CONFIG'] = 'config_segmentation.ini'
!python auto_train_segmentation/train.py train-model --lr 4.365158267319202e-05

INFO     - 
CLASSES INFERRED FROM balloon/data/balloon/result.json: <ClassMap: {'background': 0, 'balloon': 1}> | __main__:<module>:34
100%|███████████████████████████████████████| 255/255 [00:00<00:00, 2244.67it/s]
INFO     - Autofixing records | icevision.parsers.parser:parse:136
100%|████████████████████████████████████████| 13/13 [00:00<00:00, 13218.41it/s]
INFO     - Loaded weights from baloons/models/model.pt to given model | __main__:train_model:87
[0, 1.078104853630066, 0.80428147315979, 0.15314963017105154, '00:07']
[0, 1.056432843208313, 0.809590756893158, 0.16396020767439654, '00:08']
[1, 1.0458848476409912, 0.8110312819480896, 0.1521259160799801, '00:08']
[2, 1.047467589378357, 0.8237537741661072, 0.13200266981122477, '00:08']
[3, 1.0257574319839478, 0.817683219909668, 0.13218423967902862, '00:07']
[4, 1.028222680091858, 0.8430755138397217, 0.12064020289547524, '00:08']
[5, 1.0255986452102661, 0.8170281052589417, 0.13825889154935847, '00:07']
[6, 1.0311779975891113, 0.81270

---
## Validate

In [ ]:
!python auto_train_segmentation/infer.py balloon/data/balloon/validation